In [2]:
from StatsAggregation.common import get_data_for_df
import sys
import unicodedata
import os
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import pymorphy2

In [3]:
writers_dir = input()

C:\Users\annag\Documents\MLDS_writers


In [5]:
data = {}
for writer in os.listdir(writers_dir):
    texts = get_data_for_df(writer, writers_dir)
    data[writer] = texts

In [7]:
for k, v in data.items():
    print(f'{k}: {len(v)}')

Аверченко: 1482
Булгаков: 1629
Бунин: 1335
Гоголь: 445
Горький: 1682
Достоевский: 4052
Карамзин: 42
Куприн: 1368
Лермонтов: 134
Мордовцев: 772
Пушкин: 564
Салтыков-Щедрин: 1373
Толстой: 2217
Тургенев: 350
Чехов: 291


In [8]:
pre_df = []
for k in data:
    for book, text in data[k]:
        pre_df.append([k, book, text])
df = pd.DataFrame(pre_df, columns=['author', 'book', 'text'])

In [9]:
df.head()

,author,book,text
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет. Материалы [к биографии]\nКак б...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж, — качал головой сдержанный Ре-ми. — Н..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь, что нами за эти пять ле..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8\nАверченко А. Избранные рассказы. М., 1985. ..."


In [10]:
df.to_csv('author_text.csv', index=False)

In [11]:
punkt = ''.join([chr(i) for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')])

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\annag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def remove_punkt(text):
    tokens = word_tokenize(text, language='russian')
    tokens = [token.strip(punkt) for token in tokens]
    return ' '.join([token for token in tokens if token])

In [14]:
df_no_punkt = df.copy()

df_no_punkt['text_no_punkt'] = df_no_punkt['text'].apply(remove_punkt)
df_no_punkt.drop('text', axis=1, inplace=True)

In [15]:
df_no_punkt.head()

,author,book,text_no_punkt
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет Материалы к биографии Как будто...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Да уж качал головой сдержанный Ре-ми Нехорошо ...
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Поверит ли кто-нибудь что нами за эти пять лет...
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,8 Аверченко А Избранные рассказы М 1985 С 7 9 ...


In [16]:
df_no_punkt.to_csv('author_text_no_punkt.csv', index=False)

In [24]:
def lemmatize_with_tags(text):
    sw = set(stopwords.words('russian')) | set(df['author'].str.lower().unique())
    morph = pymorphy2.MorphAnalyzer()
    lemmas = []
    tags = []
    for w in text.split():
        anls = morph.parse(w)[0]
        if anls.normal_form not in sw:
            lemmas.append(anls.normal_form)
            tags.append(f'{len(w)}_{anls.tag.POS}')
    return ' '.join(lemmas), ' '.join(tags)

In [25]:
lemmas_and_tags = list(map(lemmatize_with_tags, df_no_punkt['text_no_punkt']))

In [26]:
df_lemmas_and_tags = pd.DataFrame(lemmas_and_tags, columns=['lemmas', 'tags'])
df_lemmas_and_tags['author'] = df['author']
df_lemmas_and_tags['book'] = df['book']
df_lemmas_and_tags = df_lemmas_and_tags[['author', 'book', 'lemmas', 'tags']]

In [27]:
df_lemmas_and_tags.head()

,author,book,lemmas,tags
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,пять год материал биография кроваво-красный ра...,4_NUMR 3_NOUN 9_NOUN 9_NOUN 15_ADJF 6_NOUN 8_V...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,качать голова сдержать ре-ми нехорошо нехорошо...,5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_ADVB 4_A...
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,поверить кто-нибудь пять год совместно м.г кор...,7_VERB 10_NPRO 4_NUMR 3_NOUN 9_ADVB 3_None 11_...
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,8 избранный рассказ м 1985 7 9 ежегодник 156 1...,1_None 9_ADJF 8_NOUN 1_NOUN 4_None 1_None 1_No...


In [28]:
df_lemmas_and_tags.to_csv('author_lemmas_tags.csv', index=False)

In [29]:
df_tokens = df.copy()

df_tokens['tokens'] = [' '.join(word_tokenize(text, language='russian')) for text in df['text']]
df_tokens.drop('text', axis=1, inplace=True)

In [31]:
df_tokens.head()

,author,book,tokens
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."


In [32]:
df_tokens.to_csv('author_tokens.csv', index=False)

In [33]:
aggregated_df = pd.concat([df[['author', 'book']]] + [x.iloc[:, 2:] for x in (df, df_no_punkt, df_lemmas_and_tags, df_tokens)], axis=1)

In [34]:
aggregated_df.head()

,author,book,text,text_no_punkt,lemmas,tags,tokens
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет. Материалы [к биографии]\nКак б...,Мы за пять лет Материалы к биографии Как будто...,пять год материал биография кроваво-красный ра...,4_NUMR 3_NOUN 9_NOUN 9_NOUN 15_ADJF 6_NOUN 8_V...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...,Сидел он за большим письменным столом перед де...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж, — качал головой сдержанный Ре-ми. — Н...",Да уж качал головой сдержанный Ре-ми Нехорошо ...,качать голова сдержать ре-ми нехорошо нехорошо...,5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_ADVB 4_A...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь, что нами за эти пять ле...",Поверит ли кто-нибудь что нами за эти пять лет...,поверить кто-нибудь пять год совместно м.г кор...,7_VERB 10_NPRO 4_NUMR 3_NOUN 9_ADVB 3_None 11_...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8\nАверченко А. Избранные рассказы. М., 1985. ...",8 Аверченко А Избранные рассказы М 1985 С 7 9 ...,8 избранный рассказ м 1985 7 9 ежегодник 156 1...,1_None 9_ADJF 8_NOUN 1_NOUN 4_None 1_None 1_No...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."


In [35]:
aggregated_df.to_csv('aggregated_df.csv', index=False)